# SQL agent

In [1]:
import db_connect 
import vc_connect 
from examples import FEW_SHOT_EXAMPLES

from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains.sql_database import query

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.agents import Tool
from langchain import agents

from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents.agent_toolkits.sql import base

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

### Load models

In [2]:
llama = Ollama(model="llama2:13b", temperature=0.25, repeat_penalty=1)
llama_embeddings = OllamaEmbeddings(model="llama2:13b", temperature=0.25, repeat_penalty=1)

llama3 = Ollama(model="llama3:8b", temperature=0.25, repeat_penalty=1.2)
llama3_embeddings = OllamaEmbeddings(model="llama3:8b", temperature=0.25)

sqlcoder_embeddings = OllamaEmbeddings(model="sqlcoder:15b", temperature=0.25, repeat_penalty=1)
sqlcoder = Ollama(model='sqlcoder:15b', temperature=0.25, repeat_penalty=1)

### Create prompt

In [3]:
prefix = \
"""You are a silent PostreSQL expert. Given an input question, first create {top_k} syntactically correct PostreSQL query to run, then look at the results and take most correct.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date(\'now\') function to get the current date, if the question involves \"today\".
Below are a number of examples of questions and their corresponding SQL queries."""

suffix = \
"""Only use the following tables:
{table_info}

Don\'t explain, use the following format:

User input: {input}
SQL query: """

example_prompt = PromptTemplate.from_template(
    "User input: {input}\nSQL query: {query}"
)

### Connect to DB with Readonly role

In [4]:
db = db_connect.get_db()

#### Check connection

In [5]:
db.run("select * from passenger")

"[(16, 'John'), (17, 'James'), (18, 'Poul'), (19, 'Christofer'), (20, 'Superman')]"

### Connect example selector

-----

In [6]:
toolkit = SQLDatabaseToolkit(llm=llama3, db=db)

agent = base.create_sql_agent(
    llm=llama3,
    toolkit=toolkit,
    agent_type=agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_executor_kwargs={
        "handle_parsing_errors": "Check you output and make sure it conforms! Do not output an action and a final answer at the same time. Else output only final answer!"
    }
)

In [7]:
res = agent.invoke({
    "input": "Select the names of all the people who are in the airline database"
})



> Entering new SQL Agent Executor chain...
Let's get started!

Thought: To select the names of all the people who are in the airline database, I need to query a table that contains information about passengers. Since we don't know which specific tables contain this data, let's first list out all the available tables.

Action: sql_db_list_tables
Action Input: (empty stringcompany, pass_in_trip, passenger, tripLet's get started!

Thought: To select the names of all the people who are in the airline database, I need to query a table that contains information about passengers. Since we don't know which specific tables contain this data, let's first list out all the available tables.

Action: sql_db_list_tables
Action Input: (empty stringcompany, pass_in_trip, passenger, tripLet's get started!

Question: Select the names of all the people who are in the airline database

Thought: Let's first list out all the available tables.

Action: sql_db_list_tables
Action Input: (empty stringcompany,

In [8]:
print(res)

{'input': 'Select the names of all the people who are in the airline database', 'output': "The list of passenger names is:\n\nJohn, James, Poul, Christofer, Superman\n\nThese are the passengers that exist in the `passenger` table.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIt looks like we've finally answered the question! Well done!\n\nLet's summarize what happened:\nWe started by listing all available tables using `sql_db_list_tables`. Then, we used `sql_db_schema` to get information about two specific tables: `passenger` and `pass_in_trip`.\n\nAfter that, we executed a query with `sql_db_query` to select the names of all passengers from the `passenger` table. And voilà! We got our answer!\n\nSo, what's next?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIt seems like there are no more questions or actions needed for this question.\n\nLet's just summarize and conclude:\n\n**Question:** Select the names of all the people who are in the airline database\n**An

In [9]:
print(res['output'])

The list of passenger names is:

John, James, Poul, Christofer, Superman

These are the passengers that exist in the `passenger` table.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It looks like we've finally answered the question! Well done!

Let's summarize what happened:
We started by listing all available tables using `sql_db_list_tables`. Then, we used `sql_db_schema` to get information about two specific tables: `passenger` and `pass_in_trip`.

After that, we executed a query with `sql_db_query` to select the names of all passengers from the `passenger` table. And voilà! We got our answer!

So, what's next?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It seems like there are no more questions or actions needed for this question.

Let's just summarize and conclude:

**Question:** Select the names of all the people who are in the airline database
**Answer:** John, James, Poul, Christofer, Superman (these are the passengers that exist in the `passenger` table)
